Sample Questions:

    Perform a t-test to compare income between genders.

    Is there a significant relationship between education and employment?

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(data[data['gender']=='M']['income'],
          data[data['gender']=='F']['income'], nan_policy='omit')


Is there a significant difference in income between males and females?

In [ ]:
from scipy.stats import ttest_ind

male_income = data[data['gender'] == 'Male']['income'].dropna()
female_income = data[data['gender'] == 'Female']['income'].dropna()

t_stat, p_val = ttest_ind(male_income, female_income)
print(f"T-statistic: {t_stat:.3f}, P-value: {p_val:.3f}")

#📌 Interpretation:
#If p < 0.05 → statistically significant difference in mean income by gender.

 ## 2. ANOVA: Compare More than Two Means

Task:
Assume we bin education_score into low, medium, high. Is there a difference in income across these groups?

In [ ]:
import pandas as pd
from scipy.stats import f_oneway

# Binning education_score
data['edu_level'] = pd.qcut(data['education_score'], q=3, labels=['Low', 'Medium', 'High'])

# Perform ANOVA
groups = [group['income'].dropna() for name, group in data.groupby('edu_level')]
f_stat, p_val = f_oneway(*groups)
print(f"F-statistic: {f_stat:.3f}, P-value: {p_val:.3f}")


#📌 Interpretation:
#p < 0.05 means income differs across education levels.

## 3. Chi-Square Test: Test Association Between Categorical Variables

Task:
Is there a significant association between gender and target?

In [ ]:
from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(data['gender'], data['target'])
chi2, p_val, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-square: {chi2:.3f}, P-value: {p_val:.3f}")

#p < 0.05 → gender and target are associated.

## 4. Correlation Test: Linear Relationship Between Two Continuous Variables

Task:
Is there a significant correlation between income and education_score?

In [ ]:
from scipy.stats import pearsonr

corr, p_val = pearsonr(data['income'].dropna(), data['education_score'].dropna())
print(f"Correlation: {corr:.3f}, P-value: {p_val:.3f}")

#Correlation near ±1 = strong linear relationship.

#p < 0.05 → the correlation is statistically significant.

## 5. Bonus: Interpret a P-Value

Question:

# In a test comparing two means, you get a p-value of 0.01. What does this mean?

📌 Answer (written):
A p-value of 0.01 means there is a 1% probability that the observed difference (or a more extreme one) could occur by random chance if the null hypothesis is true. Since this is less than 0.05, we reject the null hypothesis and conclude the difference is statistically significant.


✅ 1. T-Test Interpretation

# You performed a t-test comparing income between males and females and obtained a p-value of 0.03. What does this result suggest?

Sample Answer:
A p-value of 0.03 indicates that there is only a 3% chance that the observed difference in income between males and females occurred by random chance. Since the p-value is below the typical threshold of 0.05, we reject the null hypothesis and conclude that there is a statistically significant difference in income between genders in our sample.
🧪 2. ANOVA Interpretation

# An ANOVA test comparing income across three education levels gave an F-statistic of 5.23 and a p-value of 0.007. What can we conclude?

Sample Answer:
The p-value of 0.007 is less than 0.05, suggesting that there is a statistically significant difference in average income across the three education level groups. However, ANOVA does not tell us which specific groups differ — for that, we would need to conduct a post-hoc test such as Tukey's HSD.
🔗 3. Chi-Square Interpretation

# You run a chi-square test between gender and target variable (0/1 outcome), and obtain a p-value of 0.12. What does this mean?

Sample Answer:
A p-value of 0.12 is greater than the common significance level of 0.05, meaning we fail to reject the null hypothesis. Therefore, we do not have enough evidence to conclude that there is an association between gender and the target outcome in this sample.
📈 4. Correlation Interpretation

# You find a Pearson correlation of 0.65 between education_score and income, with a p-value of 0.0001. What does this tell you?

Sample Answer:
A correlation coefficient of 0.65 indicates a moderately strong positive linear relationship between education score and income: as education score increases, income tends to increase as well. The very small p-value suggests this correlation is statistically significant and unlikely to be due to random chance.
❓ 5. General: What Is a P-Value?

# What is a p-value, and how should it be interpreted?

Sample Answer:
A p-value is the probability of observing data as extreme as, or more extreme than, what we observed, assuming the null hypothesis is true. A small p-value (typically < 0.05) suggests that the observed result is unlikely under the null hypothesis and provides evidence to reject it. However, it does not measure effect size or practical importance.

In [ ]:
df = df.rename(columns={
    'old_name1': 'new_name1',
    'old_name2': 'new_name2'
})


## Detect Outliers, Add Flags to Original Data, Extract All Rows with Any Outlier

In [ ]:
def detect_outliers_iqr(df):
    outlier_flags = pd.DataFrame(index=df.index)
    numeric_cols = df.select_dtypes(include='number').columns
    
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        outlier_flags[col + '_outlier'] = ~df[col].between(lower, upper)
    
    return outlier_flags

#Add Flags to Original Data
outlier_flags = detect_outliers_iqr(df)
df_with_outliers = pd.concat([df, outlier_flags], axis=1)

#Extract All Rows with Any Outlier
# Get rows where at least one outlier is present
outliers_only = df_with_outliers[outlier_flags.any(axis=1)]

#Save Outliers to Excel Sheet
outliers_only.to_excel("outliers_detected.xlsx", index=False)


## Python Code: IQR Outlier Detection for All Numeric Columns

In [ ]:
import pandas as pd

def detect_outliers_iqr(df):
    outlier_flags = pd.DataFrame(index=df.index)
    
    # Only apply to numeric columns
    numeric_cols = df.select_dtypes(include='number').columns
    
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        
        # Create a boolean mask for outliers
        outlier_flags[col + '_outlier'] = ~df[col].between(lower, upper)
    
    return outlier_flags


# Apply to your cleaned data
outliers_df = detect_outliers_iqr(df)

# Count outliers per column
outliers_df.sum()

#Add Outlier Info to Original DataFrame

df_with_flags = pd.concat([df, detect_outliers_iqr(df)], axis=1)

#Remove All Outliers

mask = ~detect_outliers_iqr(df).any(axis=1)
df_no_outliers = df[mask]
